In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [6]:
%%time 
train_data = pd.read_csv("data/ohe_train_recipes_v2.csv",index_col="id")
test_data = pd.read_csv("data/ohe_test_recipes_v2.csv",index_col="id")

CPU times: user 13 s, sys: 1.6 s, total: 14.6 s
Wall time: 15.3 s


In [7]:
train_data.head(2)

,1% buttermilk,1% chocolate milk,1% cottage cheese,1% milk,"2 1/2 to 3 lb. chicken, cut into serving pieces",2% cottage cheese,2% low fat cheddar chees,2% lowfat greek yogurt,2% milk mozzarella cheese,2% reduced-fat milk,...,yuzu,yuzu juice,za'atar,zest,zesty italian dressing,zinfandel,ziti,zucchini,zucchini blossoms,cuisine
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,spanish
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mexican


In [8]:
X=train_data.drop(['cuisine'],axis=1)
response=train_data['cuisine']

In [9]:
X_train,X_validate,y_train,y_validate=train_test_split(X,response,test_size=0.5,random_state=22)

In [10]:
X_train.shape[0], X_validate.shape[0], X_validate.shape[0]/(X_validate.shape[0]+X_train.shape[0])

(19887, 19887, 0.5)

In [32]:
%%time
lr=LogisticRegression(C=0.05)
lr.fit(X_train,y_train)

CPU times: user 3min 7s, sys: 1.95 s, total: 3min 9s
Wall time: 26.8 s


LogisticRegression(C=0.05)

In [33]:
lr.score(X_validate,y_validate), lr.score(X_train,y_train)

(0.7096595766078343, 0.7479760647659275)

In [8]:
from sklearn.preprocessing import StandardScaler

In [10]:
scaler = StandardScaler()
X_train_trans = scaler.fit_transform(X_train)

In [11]:
X_validate_trans = scaler.transform(X_validate)

In [12]:
%%time
lr=LogisticRegression()
lr.fit(X_train_trans,y_train)

CPU times: user 6min 59s, sys: 4.02 s, total: 7min 3s
Wall time: 59 s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
lr.score(X_validate,y_validate), lr.score(X_train,_train)

0.6918744971842317

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
parameters = {'C':[0.05,0.1,0.5,1,2]}
lr=LogisticRegression()

In [36]:
clf = GridSearchCV(lr, parameters)

In [37]:
clf.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

KeyboardInterrupt: 

In [11]:
clustering = KMeans(init='k-means++', n_clusters=20)


In [12]:
clustering.fit(X_train)

KMeans(n_clusters=20)

In [17]:
clusters_X_train    = pd.Series(clustering.predict(X_train),   name="AutoCluster",index=X_train.index)
clusters_X_validate = pd.Series(clustering.predict(X_validate),name="AutoCluster",index=X_validate.index)

In [19]:
X_train_w_clusters = X_train.merge(clusters_X_train, left_index=True, right_index=True)
X_validate_w_clusters = X_validate.merge(clusters_X_validate, left_index=True, right_index=True)

In [20]:
%%time
lr=LogisticRegression()
lr.fit(X_train_w_clusters,y_train)

CPU times: user 11min 42s, sys: 3.91 s, total: 11min 45s
Wall time: 1min 49s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:
%%time
lr.score(X_train_w_clusters,y_train), lr.score(X_validate_w_clusters,y_validate)

CPU times: user 7.4 s, sys: 1.37 s, total: 8.77 s
Wall time: 2.51 s


(0.7753809020968472, 0.7330416855232061)